In [1]:
import pandas as pd
import numpy as np
import chardet as chardet
        

In [2]:

with open('Data/BHCF20250331.txt', 'rb') as f:
        raw_data = f.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
        print(f"Detected encoding: {encoding}")

Detected encoding: ascii


In [3]:
raw_data = open('Data/BHCF20250331.txt', 'rb').read()
result = chardet.detect(raw_data)
encoding = result['encoding']
print(f"Detected encoding: {encoding}")

Detected encoding: ascii


In [18]:
import glob as glob
all_files = glob.glob('Data/BHCF*.txt')
all_files_2008 = glob.glob('Data_Processing/inputs/FED_data/BHCF200[7-9]*.txt')
print(all_files_2008)

df_list = []
for file_path in all_files_2008:
    print(file_path)
    raw_data = open(file_path, 'rb').read()
    result = chardet.detect(raw_data)
    encoding = result['encoding']
    df = pd.read_csv(file_path, delimiter = '^', encoding=encoding, engine='python', on_bad_lines='skip')

    #Potential Row Level Filtering on Tier 1 Capital Ratio
    df = df[df['BHCA7206'].isna() == False]

    df_list.append(df)
    print(encoding,df.shape)
combined_df = pd.concat(df_list, ignore_index=True)

[]


ValueError: No objects to concatenate

In [21]:
from pathlib import Path

base = Path("Data_Processing/inputs/FED_data")

files = []
for year in range(2007, 2011):  # 2007–2010 inclusive
    pattern = f"BHCF{year}*/*.txt"
    files.extend(base.glob(pattern))

# If you want a sorted list:
files = sorted(files)

# Print or inspect
#for f in files:
    #print(f)

df_list = []
for file_path in files:
    print(file_path)
    raw_data = open(file_path, 'rb').read()
    result = chardet.detect(raw_data)
    encoding = result['encoding']
    df = pd.read_csv(file_path, delimiter = '^', encoding=encoding, engine='python', on_bad_lines='skip')

    #Potential Row Level Filtering on Tier 1 Capital Ratio
    #df = df[df['BHCA7206'].isna() == False]

    df_list.append(df)
    print(encoding,df.shape)
combined_df = pd.concat(df_list, ignore_index=True)

Data_Processing/inputs/FED_data/BHCF20070331/BHCF20070331.txt
ascii (1218, 1476)
Data_Processing/inputs/FED_data/BHCF20070630/BHCF20070630.txt
ascii (5688, 1477)
Data_Processing/inputs/FED_data/BHCF20070930/BHCF20070930.txt
ascii (1208, 1477)
Data_Processing/inputs/FED_data/BHCF20071231/BHCF20071231.txt
ascii (5670, 1477)
Data_Processing/inputs/FED_data/BHCF20080331/BHCF20080331.txt
ascii (1218, 1639)
Data_Processing/inputs/FED_data/BHCF20080630/BHCF20080630.txt
ascii (5602, 1642)
Data_Processing/inputs/FED_data/BHCF20080930/BHCF20080930.txt
ascii (1217, 1642)
Data_Processing/inputs/FED_data/BHCF20081231/BHCF20081231.txt
ascii (5537, 1642)
Data_Processing/inputs/FED_data/BHCF20090331/BHCF20090331.txt
ascii (1281, 1670)
Data_Processing/inputs/FED_data/BHCF20090630/BHCF20090630.txt
ascii (5510, 1924)
Data_Processing/inputs/FED_data/BHCF20090930/BHCF20090930.txt
ascii (1265, 1926)
Data_Processing/inputs/FED_data/BHCF20091231/BHCF20091231.txt
ascii (5434, 1927)
Data_Processing/inputs/FED_d

In [22]:
print(combined_df.shape)

(54024, 2047)


In [23]:
dddf = pd.read_csv('/Users/jacksonlipfert/Downloads/MDRM/MDRM_CSV.csv')

In [24]:
#dddf.head()
short_dddf = dddf[['Mnemonic','Item Code','Item Name']]


short_dddf['Full Code'] = short_dddf['Mnemonic'] + short_dddf['Item Code']
important_codes = ['RSSD', 'BHBC', 'BHCA', 'BHCB', 'BHCK', 'BHCM', 'BHCP', 'BHCT', 'BHCW',
       'BHCX', 'BHCY', 'BHDM', 'BHFN', 'BHOD', 'BHPA', 'BHPX', 'BHSP', 'BHSX',
       'BHTX', 'TEXT']
short_dddf = short_dddf[short_dddf['Mnemonic'].isin(important_codes)]


/var/folders/17/1lp6v8rx7hg46v35nrbf4mk00000gn/T/ipykernel_3096/756459364.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_dddf['Full Code'] = short_dddf['Mnemonic'] + short_dddf['Item Code']


In [25]:
# create mapper


column_mapper = {}

#Columns that appear across multiple types of companies with the same balance sheet name, need to keep code to disambiguate
special_columns = [
    'TOTAL ASSETS',
    'TOTAL EQUITY',
    'TOTAL EQUITY CAPITAL',
    'NET INCOME',
    'NET INCOME (LOSS)'
    ]

def get_name_from_full_code(fullcode, df):
    '''get column name from full code found in data dictionary'''
    column = df['Item Name'][df['Full Code'] == fullcode]
    col_name = column.iloc[0]
    return col_name


#Loop over short data dictionary and populate column mapper with name
for code in short_dddf['Full Code']:
    column_mapper[code] = get_name_from_full_code(code,short_dddf)


def disambiguate_column_mapper(column_mapper, special_columns):
    '''Loop over column mapper and replace ambigious columns (found in special_columns)'''

    for key,value in column_mapper.items():
        if value in special_columns:
            column_mapper[key] = f'{value}: {key}' 
    return column_mapper

def disambiguate_all_column_mapper(column_mapper):
    '''Loop over column mapper and replace ambigious columns (found in special_columns)'''

    for key,value in column_mapper.items():
        column_mapper[key] = f'{value}: {key}' 
    return column_mapper

#Final column mapper
#disambig_column_mapper = disambiguate_column_mapper(column_mapper, special_columns)
disambig_column_mapper = disambiguate_all_column_mapper(column_mapper)

#check to make sure columns have been renamed
column_name_in_question = 'TIER 1 RISK-BASED CAPITAL RATIO'
for key,value in disambig_column_mapper.items():
    if column_name_in_question in str(value):
        print(value)
    

    

TIER 1 RISK-BASED CAPITAL RATIO: BHCA7206
TIER 1 RISK-BASED CAPITAL RATIO: BHCK7206
TIER 1 RISK-BASED CAPITAL RATIO: BHCW7206


In [26]:
renamed_combined_df = combined_df.rename(columns=disambig_column_mapper)
print(renamed_combined_df.shape)

(54024, 2047)


In [14]:
search_string = 'SHORT'
matching_columns = renamed_combined_df.columns[renamed_combined_df.columns.str.contains(search_string)]
print(matching_columns)

Index(['LIABILITY FOR SHORT POSITIONS: EQUITY SECURITIES: BHCKG209',
       'LIABILITY FOR SHORT POSITIONS: DEBT SECURITIES: BHCKG210',
       'LIABILITY FOR SHORT POSITIONS: ALL OTHER ASSETS: BHCKG211',
       'CASH FLOWS FROM FINANCING ACTIVITIES: NET CHANGE IN PURCHASED FUNDS AND OTHER SHORT-TERM BORROWINGS: BHCPF818',
       'SHORT-TERM BORROWINGS INCLUDED IN ITEM 3621 FROM PARENT BANK HOLDING COMPANY(S): BHSP3524',
       'SHORT-TERM BORROWINGS INCLUDED IN ITEM 3621 FROM SUBSIDIARY BANK HOLDING COMPANY(S): BHSP3526',
       'ENTITY SHORT NAME: RSSD9010'],
      dtype='object')


In [15]:

col = 'REPORTING DATE (CC;YR;MO;DA): RSSD9999'
s = renamed_combined_df[col]

mask = s.floordiv(10_000) == 2025          # works with nullable Int64 too
out = renamed_combined_df['ENTITY SHORT NAME: RSSD9010'][mask]
print(len(out.unique()))

354


In [27]:
renamed_combined_df.to_csv('2007-2010_Bank_Data.csv', index=False)